# RANZCR_Ensemble
![](https://storage.googleapis.com/kaggle-competitions/kaggle/23870/logos/header.png?t=2020-12-01-04-28-05)

Serious complications can occur as a result of malpositioned lines and tubes in patients. Doctors and nurses frequently use checklists for placement of lifesaving equipment to ensure they follow protocol in managing patients. Yet, these steps can be time consuming and are still prone to human error, especially in stressful situations when hospitals are at capacity.

Hospital patients can have catheters and lines inserted during the course of their admission and serious complications can arise if they are positioned incorrectly. Nasogastric tube malpositioning into the airways has been reported in up to 3% of cases, with up to 40% of these cases demonstrating complications [1-3]. Airway tube malposition in adult patients intubated outside the operating room is seen in up to 25% of cases [4,5]. The likelihood of complication is directly related to both the experience level and specialty of the proceduralist. Early recognition of malpositioned tubes is the key to preventing risky complications (even death), even more so now that millions of COVID-19 patients are in need of these tubes and lines.

The gold standard for the confirmation of line and tube positions are chest radiographs. However, a physician or radiologist must manually check these chest x-rays to verify that the lines and tubes are in the optimal position. Not only does this leave room for human error, but delays are also common as radiologists can be busy reporting other scans. Deep learning algorithms may be able to automatically detect malpositioned catheters and lines. Once alerted, clinicians can reposition or remove them to avoid life-threatening complications.

## This ensemble includes models from 3 parts of the project:
- Xception part(`ranzcr-xception-tpu-baseline.ipynb`, `ranzcr-xception-tpu-prediction.ipynb`)
- EfficientNetB7 part(`effnetb7-tpu.ipynb`, `effnetb7-tpu-prediction.ipynb`)
- EfficientNetB4_CV part(`ranzcr-efb4-cv-tr.ipynb`, `ranzcr-efb4-cv-pr.ipynb`)

In [1]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

In [2]:
import efficientnet.tfkeras as efn

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import Xception, EfficientNetB7, InceptionResNetV2
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

import os, cv2
from PIL import Image

In [4]:
# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print(f'Running on TPU {tpu.master()}')
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy()

# AUTO = tf.data.experimental.AUTOTUNE
# REPLICAS = strategy.num_replicas_in_sync
# print(f'REPLICAS: {REPLICAS}')

In [5]:
WORK_DIR = '../input/ranzcr-clip-catheter-line-classification'
os.listdir(WORK_DIR)

['train_tfrecords',
 'sample_submission.csv',
 'train_annotations.csv',
 'test_tfrecords',
 'train.csv',
 'test',
 'train']

In [6]:
# Data
train = pd.read_csv(os.path.join(WORK_DIR, "train.csv"))
train_images = WORK_DIR + "/train/" + train['StudyInstanceUID'] + '.jpg'

ss = pd.read_csv(os.path.join(WORK_DIR, 'sample_submission.csv'))
test_images = WORK_DIR + "/test/" + ss['StudyInstanceUID'] + '.jpg'

label_cols = ss.columns[1:]
labels = train[label_cols].values

train_annot = pd.read_csv(os.path.join(WORK_DIR, "train_annotations.csv"))

print('Labels:\n', '*'*20, '\n', label_cols.values)
print('*'*50)
train.head()

Labels:
 ******************** 
 ['ETT - Abnormal' 'ETT - Borderline' 'ETT - Normal' 'NGT - Abnormal'
 'NGT - Borderline' 'NGT - Incompletely Imaged' 'NGT - Normal'
 'CVC - Abnormal' 'CVC - Borderline' 'CVC - Normal'
 'Swan Ganz Catheter Present']
**************************************************


,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1


In [7]:
# Main parameters
BATCH_SIZE = 8 * 1
STEPS_PER_EPOCH = len(train) * (1 / 7 * 6) / BATCH_SIZE
VALIDATION_STEPS = len(train) * (1 / 7 * 1) / BATCH_SIZE
EPOCHS = 30
TARGET_SIZE_1 = 550
TARGET_SIZE_2 = 750

# EffNetB7

In [8]:
def build_decoder(with_labels = True,
                  target_size = (TARGET_SIZE_2, TARGET_SIZE_2), 
                  ext = 'jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels = True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        
        p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
        p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
        p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
        p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#         p_crop = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
        
        if p_rotate > .75:
            img = tf.image.rot90(img, k = 3) # rotate 270º
        elif p_rotate > .5:
            img = tf.image.rot90(img, k = 2) # rotate 180º
        elif p_rotate > .25:
            img = tf.image.rot90(img, k = 1) # rotate 90º
            
        if p_pixel_1 >= .6:
            img = tf.image.random_saturation(img, lower = 0.75, upper = 1.25)
        if p_pixel_2 >= .6:
            img = tf.image.random_contrast(img, lower = 0.75, upper = 1.25)
        if p_pixel_3 >= .4:
            img = tf.image.random_brightness(img, max_delta = 0.1)
        
#         if p_crop > .7:
#             if p_crop > .9:
#                 img = tf.image.central_crop(img, central_fraction=.75)
#             elif p_crop > .8:
#                 img = tf.image.central_crop(img, central_fraction=.85)
#             else:
#                 img = tf.image.central_crop(img, central_fraction=.95)
#         elif p_crop > .4:
#             crop_size = tf.random.uniform([], int(TARGET_SIZE * .85), TARGET_SIZE, dtype = tf.int32)
#             img = tf.image.random_crop(img, size = [crop_size, crop_size, 3])
        
#         img = tf.image.resize(img, size = [TARGET_SIZE, TARGET_SIZE])
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = 32, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [9]:
test_df_B7 = build_dataset(
    test_images, bsize = BATCH_SIZE, repeat = False, 
    shuffle = False, augment = False, cache = False)
test_df_B7

<PrefetchDataset shapes: (None, 750, 750, 3), types: tf.float32>

# Xception

In [10]:
def build_decoder(with_labels = True,
                  target_size = (TARGET_SIZE_2, TARGET_SIZE_2), 
                  ext = 'jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels = True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.adjust_brightness(img, 0.1)
        
#         rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)      
#         if rotate > .75:
#             img = tf.image.rot90(img, k = 3)
#         elif rotate > .5:
#             img = tf.image.rot90(img, k = 2)
#         elif rotate > .25:
#             img = tf.image.rot90(img, k = 1)
        
#         saturation = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#         if saturation >= .5:
#             img = tf.image.random_saturation(img, lower = 0.9, upper = 1.1)
        
#         contrast = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#         if contrast >= .5:
#             img = tf.image.random_contrast(img, lower = 0.9, upper = 1.1)
        
#         brightness = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#         if brightness >= .5:
#             img = tf.image.random_brightness(img, max_delta = 0.1)
            
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = 32, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [11]:
test_df_Xcep = build_dataset(
    test_images, bsize = BATCH_SIZE, repeat = False, 
    shuffle = False, augment = False, cache = False)
test_df_Xcep

<PrefetchDataset shapes: (None, 750, 750, 3), types: tf.float32>

# EffNetB4_CV

In [12]:
def build_decoder(with_labels = True,
                  target_size = (TARGET_SIZE_1, TARGET_SIZE_1), 
                  ext = 'jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels = True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        
        p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
        p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
        p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
        p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#         p_crop = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
        
        if p_rotate > .75:
            img = tf.image.rot90(img, k = 3) # rotate 270º
        elif p_rotate > .5:
            img = tf.image.rot90(img, k = 2) # rotate 180º
        elif p_rotate > .25:
            img = tf.image.rot90(img, k = 1) # rotate 90º
            
        if p_pixel_1 >= .6:
            img = tf.image.random_saturation(img, lower = 0.75, upper = 1.25)
        if p_pixel_2 >= .6:
            img = tf.image.random_contrast(img, lower = 0.75, upper = 1.25)
        if p_pixel_3 >= .4:
            img = tf.image.random_brightness(img, max_delta = 0.1)
        
#         if p_crop > .7:
#             if p_crop > .9:
#                 img = tf.image.central_crop(img, central_fraction=.75)
#             elif p_crop > .8:
#                 img = tf.image.central_crop(img, central_fraction=.85)
#             else:
#                 img = tf.image.central_crop(img, central_fraction=.95)
#         elif p_crop > .4:
#             crop_size = tf.random.uniform([], int(TARGET_SIZE * .85), TARGET_SIZE, dtype = tf.int32)
#             img = tf.image.random_crop(img, size = [crop_size, crop_size, 3])
        
#         img = tf.image.resize(img, size = [TARGET_SIZE, TARGET_SIZE])
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = 32, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [13]:
test_df_B4 = build_dataset(
    test_images, bsize = BATCH_SIZE, repeat = False, 
    shuffle = False, augment = False, cache = False)
test_df_B4

<PrefetchDataset shapes: (None, 550, 550, 3), types: tf.float32>

# EffNetB7_architecture

In [14]:
# def create_model():
#     conv_base = efn.EfficientNetB7(include_top = False, weights = 'imagenet',
#                                    input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
#     model = conv_base.output
#     model = layers.GlobalAveragePooling2D()(model)
#     model = layers.Dropout(0.5)(model)
#     model = layers.Dense(11, activation = "sigmoid")(model)
#     model = models.Model(conv_base.input, model)

#     model.compile(optimizer = Adam(lr = 0.00025),
#                   loss = [tfa.losses.SigmoidFocalCrossEntropy(alpha = 0.5, gamma = 2)],
#                   metrics = [tf.keras.metrics.AUC(multi_label = True)])
#     return model

# Xception_architecture

In [15]:
# def create_model():
#     conv_base = Xception(include_top = False, weights = 'imagenet',
#                          input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
#     model = conv_base.output
#     model = layers.GlobalAveragePooling2D()(model)
#     model = layers.Dropout(0.3)(model)
#     model = layers.Dense(11, activation = "sigmoid")(model)
#     model = models.Model(conv_base.input, model)

#     model.compile(optimizer = Adam(lr = 0.001),
#                   loss = tfa.losses.SigmoidFocalCrossEntropy(alpha = 0.5, gamma = 2),
#                   metrics = [tf.keras.metrics.AUC(multi_label = True)])
#     return model

# EffNetB4_architecture

In [16]:
# def create_model():
#     conv_base = efn.EfficientNetB4(include_top = False, weights = None,
#                                    input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
#     model = conv_base.output
#     model = layers.GlobalAveragePooling2D()(model)
#     model = layers.Dropout(0.5)(model)
#     model = layers.Dense(11, activation = "sigmoid")(model)
#     model = models.Model(conv_base.input, model)

#     model.compile(optimizer = Adam(lr = 0.0002),
#                   loss = tfa.losses.SigmoidFocalCrossEntropy(alpha = 0.5, gamma = 2),
#                   metrics = [tf.keras.metrics.AUC(multi_label = True)])
#     return model

# Prediction

In [17]:
# B7
model_B7 = models.load_model('../input/effnetb7-tpu/Efnet_750_B7_TPU.h5')

ss[label_cols] += model_B7.predict(test_df_B7, verbose = 1) * 0.5

448/448 [==============================] - 347s 774ms/step


In [18]:
# Xcep
model_Xcep = models.load_model('../input/ranzcr-xception-tpu-baseline/Xception_750_TPU.h5')

ss[label_cols] += model_Xcep.predict(test_df_Xcep, verbose = 1) * 0.3

448/448 [==============================] - 255s 570ms/step


In [19]:
# B4_CV
model_0 = models.load_model('../input/ranzcr-efb4-cv-tr/EfB4_550_0.h5')
model_1 = models.load_model('../input/ranzcr-efb4-cv-tr/EfB4_550_1.h5')
model_2 = models.load_model('../input/ranzcr-efb4-cv-tr/EfB4_550_2.h5')
model_3 = models.load_model('../input/ranzcr-efb4-cv-tr/EfB4_550_3.h5')
model_4 = models.load_model('../input/ranzcr-efb4-cv-tr/EfB4_550_4.h5')
model_5 = models.load_model('../input/ranzcr-efb4-cv-tr/EfB4_550_5.h5')
model_6 = models.load_model('../input/ranzcr-efb4-cv-tr/EfB4_550_6.h5')

models_b4 = [model_0, model_1, model_2, model_3, model_4, model_5, model_6]

FOLDS = 7
i = 1
for model in models_b4:
    print('Predicting model #{}'.format(i))
    i += 1
    ss[label_cols] += model.predict(test_df_B4, verbose = 1) / FOLDS * 0.2

Predicting model #1
448/448 [==============================] - 241s 538ms/step
Predicting model #2
448/448 [==============================] - 243s 543ms/step
Predicting model #3
448/448 [==============================] - 243s 542ms/step
Predicting model #4
448/448 [==============================] - 244s 544ms/step
Predicting model #5
448/448 [==============================] - 243s 543ms/step
Predicting model #6
448/448 [==============================] - 245s 548ms/step
Predicting model #7
448/448 [==============================] - 245s 547ms/step


# FINAL SUBMISSION (Private Score - 0.96117)

In [20]:
ss.to_csv('submission.csv', index = False)
ss

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.094909,0.285178,0.646592,0.066704,0.129690,0.196009,0.720162,0.159397,0.333510,0.746958,0.914291
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.007270,0.006673,0.005062,0.008529,0.017918,0.005382,0.004215,0.079160,0.065045,0.941784,0.005892
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.017504,0.020111,0.012239,0.023226,0.024296,0.016679,0.022756,0.129232,0.471548,0.618678,0.008469
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.103887,0.159696,0.200652,0.162909,0.175489,0.722855,0.166577,0.306858,0.259110,0.625241,0.058964
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0.014911,0.029539,0.029826,0.016025,0.034648,0.013846,0.035396,0.114872,0.253875,0.852963,0.010548
...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.2.826.0.1.3680043.8.498.81464483108873296584...,0.038316,0.027799,0.019039,0.108240,0.166932,0.740333,0.211226,0.088519,0.216131,0.847468,0.021149
3578,1.2.826.0.1.3680043.8.498.33579133018211530710...,0.016379,0.018568,0.012727,0.023395,0.030373,0.008195,0.021875,0.486562,0.476668,0.314188,0.022009
3579,1.2.826.0.1.3680043.8.498.61472811086105902907...,0.008094,0.012083,0.009175,0.013505,0.021530,0.007464,0.011909,0.072531,0.322912,0.723325,0.006533
3580,1.2.826.0.1.3680043.8.498.19434375795525494655...,0.012428,0.024656,0.013526,0.015527,0.022521,0.014183,0.023475,0.222440,0.610449,0.349628,0.008968
